In [ ]:
# Notebook from https://medium.com/@thakermadhav/build-your-own-rag-with-mistral-7b-and-langchain-97d0c92fa146
!pip install -q torch datasets
!pip install -q accelerate==0.21.0 \
                peft==0.4.0 \
                bitsandbytes==0.40.2 \
                transformers==4.31.0 \
                trl==0.4.7
!pip install -q scipy langchain transformers playwright html2text sentence_transformers faiss-gpu
!pip install -q --upgrade git+https://github.com/huggingface/transformers

In [ ]:
!playwright install > /dev/null
!playwright install-deps > /dev/null

In [ ]:
!pip install -U tokenizers

In [ ]:
import os
import torch
from transformers import (
  AutoTokenizer, 
  AutoModelForCausalLM, 
  BitsAndBytesConfig,
  pipeline
)

from transformers import BitsAndBytesConfig




from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain


#################################################################
# Tokenizer
#################################################################

model_name="mistralai/Mistral-7B-Instruct-v0.1"
model_name="mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#################################################################
# bitsandbytes parameters
#################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    cache_dir=".",
)
#load_in_4bits=True)


In [283]:
model.config.pad_token_id = tokenizer.pad_token_id

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [284]:
import arrow
import pandas as pd
PARQUET_FILE="Accords/echantillon_public_Mathilde.parquet"

In [285]:
df_init=pd.read_parquet(PARQUET_FILE)

In [286]:
prompt_template = """

### [ROLE] Role: Vous êtes un expert juridique des accords d'entreprise sans compétence de programmation informatique, seulement la lecture et la compréhension des textes. Tu ne connais aucun langage de programmation, ni les expressions régulières.


### [INST] Instruction: Votre travail consiste à peupler et mettre à jour une base des données nécessaires pour réaliser des études économétriques poussées sur le télétravail.
        Pour cela, vous devez prendre le texte d'un accord d'entreprise dans le contexte suivant, dans lesquelles figurent les informations à jour que l'on vous fourni, et d'extraire précisément l'information spécifique dont vous avez besoin.
        Vous ne devez extraire que le nombre, et uniquement le nombre. Le nombre peut être écrit en lettres ou en chiffres. (exemple : deux ou 2 )
        Si vous ne trouvez pas la réponse dans le texte vous devez le dire et ne pas chercher à en fournir un autre nombre.
        Avant de répondre, vérifier que la réponse se trouve bien dans le texte indiqué.
        Impératif pour votre raisonnement : Déconnecter les liens entre les périodicités (semaine, mois, trimestre, année)! Faire comme si les semaines, les mois, les trimestres et les années sont des concepts indépendants : dans un mois, il y a donc 0 semaine ; dans un trimestre, 0 mois et 0 semaine; dans une année, 0 trimestre, 0 mois et 0 semaine !
        Pas de proratisation !
        Il y a cinq jours ouvrés dans une semaine, donc un nombre de jour de télétravail par semaine ne peut pas être strictement supérieur à cinq.
        Si tu donnes un nombre réel, ne donne pas la fraction mais un arrondi !
        Si le nombre de jour de télétravail annuel est inférieur à 52, ce nombre de jour de télétravail concerne du télétravail occasionnel. S'il est supérieur ou égal à 52, ce nombre concerne du télétravail régulier.
        Si le nombre de jour de télétravail trimestriel est inférieur à 13, ce nombre de jour de télétravail concerne du télétravail occasionnel. S'il est supérieur ou égal à 13, ce nombre concerne du télétravail régulier.
        Si le nombre de jour de télétravail mensuelle est inférieur à 4, ce nombre de jour de télétravail concerne du télétravail occasionnel. S'il est supérieur ou égal à 4, ce nombre concerne du télétravail régulier.
        Structure ta réponse comme suit :
        * Mon raisonnenement :
        * Phrase ou paragraphe du contexte qui répond à la question :
        * Ma réponse :
        
### Contexte: 
{context}

### QUESTION:
{question} [/INST]
"""

In [287]:
# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

# Create llm chain 
llm_chain = LLMChain(llm=mistral_llm, prompt=prompt)

In [288]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
import warnings
warnings.filterwarnings('ignore')
import re
def guess_reponse_booleenne(reponse):
    reponse_booleenne=None
    if reponse["text"].startswith("\nOui") or reponse["text"].startswith("Oui") :
        reponse_booleenne=1
    elif reponse["text"].startswith("\nNon") or reponse["text"].startswith("Non") :
        reponse_booleenne=0
    return reponse_booleenne

def guess_reponse_durée(reponse):
    reponse_booleenne=None
    if "indéterminée" in reponse["text"].lower() :
        reponse_booleenne=0
    elif "déterminée" in reponse["text"].lower():
        reponse_booleenne=1
    return reponse_booleenne

def guess_reponse_nombre(reponse,pattern="nombre"):
    reponse_nombre=None
    REGEX=rf".*{pattern}=(\d+)"
    numbers=re.findall(REGEX,reponse["text"])
    if numbers:
        reponse_nombre=numbers[0]
    return reponse_nombre

In [289]:
Q_DUREE="S'il est fait mention d'une durée de l'accord, est-ce que l'accord est à durée déterminée ou à durée indéterminée ? déterminée=1 ou indéterminée=0 ou NA?"
Q_REVERS="S'il est fait mention d'une clause et d'un article de réversibilité, est-ce que l'accord a une clause ou un article de réversibilité ? Oui=1 ou non=0 ou NA?"
Q_ADAPT="S'il est fait mention d'une période d'adaptation, est-ce que l'accord comprend une période d'adaptation ? Oui=1 ou non=0 ou NA?"
Q_TTREG="Est-ce qu'au moins un paragraphe mentionne du télétravail régulier ?"
Q_TTOCA="Est-ce qu'au moins un paragraphe mentionne du télétravail occasionnel ?"
Q_TTEXC="Est-ce qu'au moins un paragraphe mentionne du télétravail exceptionnel ?"
Q_DISPOSPERQTHENCEINTE="Est-ce qu'au moins un paragraphe mentionne des dispositifs spéciaux pour les travailleurs en situation de handicap, les femmes enceintes, les séniors ou les personnes présentant des fragilités ou des problèmes de santé ?"
Q_MONTANT_CAVIARDE="Est-ce qu'au moins un paragraphe mentionne un nombre sans le donner exactement, mais sous forme illisible ?"
Q_TELETRAVAIL_FLEX_SANS_LIMITE="Est-ce qu'au moins un paragraphe suggère une liberté totale dans le choix des jours de télétravail, hormis quelques contraintes de présence ?"
Q_TT_REG_NOMBRE_FORMULE=Q_TTREG+"Si oui, combien de formules (exemples de formule : si temps plein et temps partiel = 2, si choix entre 1 à 3 jours de télétravail régulier = 3) sont proposées pour du télétravail régulier ? Retourne à la fin 'nombre=max(tes réponses)'"
PERIODICITE={"MOIS":"mois","TRIM":"trimestre","ANNEE":"année", "SEM":"semaine"}
PERIODICITE2={"JOUR":"jour","MOIS":"mois","TRIM":"trimestre","ANNEE":"année", "SEM":"semaine"}
Q_COMPE="Est-ce qu'au moins un paragraphe mentionne une indemnité spécialement pour l'achat d'équipements ?"
Q_COMPE_NOMBRE=Q_COMPE+"Si oui, sélectionne seulement les paragraphes mentionnant une indemnité spécialement pour l'achat d'équipements et s'il existe donne le montant lié de l'indemnité d'équipement. Si oui, retourne à la fin 'nombre=max(ta réponse)'"
Q_EQUIPEMENT="Est-ce qu'au moins un paragraphe mentionne un équipement de télétravail fourni ?"
Q_COMPS="Est-ce qu'au moins un paragraphe mentionne explicitement une indemnité de sujétion (dans le cas où l'employeur impose le télétravail, ce dernier indemnise l'employé) ?"
Q_COMPS_NOMBRE=Q_COMPS+"Si oui, sélectionne seulement les paragraphes mentionnant une indemnité de sujétion dans le cas où l'employeur impose le télétravail et s'il existe donne le montant lié de l'indemnité de sujétion dans le cas où l'employeur impose le télétravail. Si oui, retourne à la fin 'nombre=max(ta réponse)'"

In [290]:
def process_model(rag_chain,num_dossier):
    dict_reponse=dict()
    dict_reponse["num_dossier"]=num_dossier
    text=""
    reponse= rag_chain.invoke(Q_DISPOSPERQTHENCEINTE)
    text+="Q_DISPOSPERQTHENCEINTE:"+reponse["text"] + "\n"
    dict_reponse["DISPOSPERQTHENCEINTE"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_MONTANT_CAVIARDE)
    text+="Q_MONTANT_CAVIARDE:"+reponse["text"] + "\n"
    dict_reponse["MONTANT_CAVIARDE"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_TELETRAVAIL_FLEX_SANS_LIMITE)
    text+="Q_TELETRAVAIL_FLEX_SANS_LIMITE:"+reponse["text"] + "\n"
    dict_reponse["TELETRAVAIL_FLEX_SANS_LIMITE"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_TT_REG_NOMBRE_FORMULE)
    text+="Q_TT_REG_NOMBRE_FORMULE:"+reponse["text"] + "\n"
    dict_reponse["TT_REG_NOMBRE_FORMULE"]=guess_reponse_nombre(reponse)
    reponse= rag_chain.invoke(Q_DUREE)
    text+="Q_DUREE:"+reponse["text"] + "\n"
    dict_reponse["DUREE"]=guess_reponse_durée(reponse)
    reponse= rag_chain.invoke(Q_REVERS)
    text+="Q_REVERS:"+reponse["text"] + "\n"
    dict_reponse["REVERS"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_ADAPT)
    text+="Q_ADAPT:"+reponse["text"] + "\n"
    dict_reponse["ADAPT"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_TTREG)
    text+="Q_TTREG:"+reponse["text"] + "\n"
    dict_reponse["TTREG"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_TTOCA)
    text+="Q_TTOCA:"+reponse["text"] + "\n"
    dict_reponse["TTOCA"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_TTEXC)
    text+="Q_TTEXC:"+reponse["text"] + "\n"
    dict_reponse["TTEXC"]=guess_reponse_booleenne(reponse)
    for (k,v) in PERIODICITE.items():
        Q_TT=f"Hors jours supplémentaires ou exceptionnels, est-ce qu'au moins un paragraphe mentionne un nombre de jour de télétravail maximum autorisé explicitement exprimé par {v} ?"
        reponse= rag_chain.invoke(Q_TT)
        text+=f"Q_TT{k}:"+reponse["text"] + "\n"
        dict_reponse[f"TT{k}"]=guess_reponse_booleenne(reponse)
        #Q_TT_NOMBRE=f"Extrait le nombre de jour de télétravail maximum autorisé explicitement exprimé par {v} seulement à partir du texte suivant : {text_reponse}"
        Q_TT_NOMBRE=Q_TT+f"Si oui, quel est le nombre de jour de télétravail supplémentaires explicitement exprimé et nécessairement et seulement exprimé par {v} pour des dispositions spéciales. Si oui, retourne à la fin 'nombre=max(tes réponses)'"
        reponse= rag_chain.invoke(Q_TT_NOMBRE)
        text+=f"Q_TT{k}_NOMBRE:"+reponse["text"] + "\n"
        dict_reponse[f"TT{k}_NOMBRE"]=guess_reponse_nombre(reponse)
    for (k,v) in PERIODICITE.items():
        Q_TTOCA_PERIOD=f"Dans le cadre du télétravail occasionnel, est-ce qu'au moins un paragraphe mentionne un nombre de jour de télétravail maximum autorisé explicitement exprimé par {v} ?"
        reponse= rag_chain.invoke(Q_TTOCA_PERIOD)
        text+=f"TTOCA{k}:"+reponse["text"] + "\n"
        dict_reponse[f"TTOCA{k}"]=guess_reponse_booleenne(reponse)
        Q_TTOCA_NOMBRE=Q_TTOCA_PERIOD+f"Si oui, quel est le nombre de jour de télétravail occasionnel explicitement exprimé et nécessairement et seulement exprimé par {v}. Si oui, retourne à la fin 'nombre=max(tes réponses)'"
        reponse= rag_chain.invoke(Q_TTOCA_NOMBRE)
        text+=f"Q_TTOCA_{k}_NOMBRE:"+reponse["text"] + "\n"
        dict_reponse[f"TTOCA_{k}_NOMBRE"]=guess_reponse_nombre(reponse)
    for (k,v) in PERIODICITE.items():
        Q_PRESJOUR=f"Est-ce qu'au moins un paragraphe mentionne un nombre de jour de présence obligatoire sur site minimum explicitement exprimé par {v} ?"
        reponse= rag_chain.invoke(Q_PRESJOUR)
        text+=f"Q_PRESJOUR{k}:"+reponse["text"] + "\n"
        dict_reponse[f"TTPRESJOUR{k}"]=guess_reponse_booleenne(reponse)
        Q_PRESJOUR_NOMBRE=Q_PRESJOUR+f"Si oui, sélectionne seulement les paragraphes mentionnant un nombre de jour de présence obligatoire sur site minimum explicitement exprimé par {v} et s'il existe donne le nombre de jour de présence obligatoire sur site minimum explicitement exprimé par {v}. Si oui, retourne à la fin 'nombre=max(tes réponses)'"
        reponse= rag_chain.invoke(Q_PRESJOUR_NOMBRE)
        text+=f"Q_PRESJOUR{k}_NOMBRE:"+reponse["text"] + "\n"
        dict_reponse[f"PRESJOUR{k}_NOMBRE"]=guess_reponse_nombre(reponse)
    for (k,v) in PERIODICITE2.items():
        Q_COMP=f"Hors indemnité d'équipement, est-ce qu'au moins un paragraphe mentionne une indemnité forfaitaire explicitement exprimée par {v} ?"
        reponse= rag_chain.invoke(Q_COMP)
        text+=f"Q_COMP{k}:"+reponse["text"] + "\n"
        dict_reponse[f"COMP{k}"]=guess_reponse_booleenne(reponse)
        Q_COMP_NOMBRE=Q_COMP+f"Si oui, sélectionne en interne seulement les paragraphes mentionnant une indemnité forfaitaire explicitement exprimé par {v}, puis s'il existe donne le montant explicitement exprimé par {v}. Si oui, retourne à la fin 'nombre=max(ta réponse)'"
        reponse= rag_chain.invoke(Q_COMP_NOMBRE)
        text+=f"Q_COMP{k}_NOMBRE:"+reponse["text"] + "\n"
        dict_reponse[f"COMP{k}_NOMBRE"]=guess_reponse_nombre(reponse)
        if "MOIS" in k:
            Q_COMP_NOMBRE_BASE=Q_COMP+f"Si oui, sélectionne en interne seulement les paragraphes mentionnant une indemnité forfaitaire explicitement exprimé par {v}, puis s'il existe plusieurs montants explicitement exprimé par {v}, par nombre de jour. Si oui, retourne à la fin 'nombre=base_mensuelle_pour_un_jour_de_teletravail(ta réponse)'"
            reponse= rag_chain.invoke(Q_COMP_NOMBRE_BASE)
            text+=f"Q_COMP{k}_BASE_NOMBRE:"+reponse["text"] + "\n"
            dict_reponse[f"COMP{k}_BASE_NOMBRE"]=guess_reponse_nombre(reponse)
    reponse= rag_chain.invoke(Q_COMPE)
    text+="Q_COMPE:"+reponse["text"] + "\n"
    dict_reponse["COMPE"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_COMPE_NOMBRE)
    text+="Q_COMPE_NOMBRE:"+reponse["text"] + "\n"
    dict_reponse["COMPE_NOMBRE"]=guess_reponse_nombre(reponse)
    reponse= rag_chain.invoke(Q_EQUIPEMENT)
    text+="Q_EQUIPEMENT:"+reponse["text"] + "\n"
    dict_reponse["EQUIPEMENT"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_COMPS)
    text+="Q_COMPS:"+reponse["text"] + "\n"
    dict_reponse["COMPS"]=guess_reponse_booleenne(reponse)
    reponse= rag_chain.invoke(Q_COMPS_NOMBRE)
    text+="Q_COMPS_NOMBRE:"+reponse["text"] + "\n"
    dict_reponse["COMPS_NOMBRE"]=guess_reponse_nombre(reponse)
    return dict_reponse,text

In [291]:
PERIODICITE={ "SEM":"semaine"}

def process_model(rag_chain,num_dossier):
    dict_reponse=dict()
    dict_reponse["num_dossier"]=num_dossier
    text=""
    for (k,v) in PERIODICITE.items():
        Q_TT_NOMBRE=f"Est-ce qu'un paragraphe comporte le mot-clé {v} ou une notion de fréquence par {v} et concerne le nombre de jour de télétravail autorisé (ne compte pas les paragraphes qui relatent d'indemnisation par jour de télétravail, ni le télétravail ponctuel ou occasionnel)?"
        Q_TT_NOMBRE+=f" SI oui, jusqu'à combien de jour de télétravail un employé peut-il télétravailler régulièrement, jour exprimé seulement par {v} et explicitement dans un des paragraphes (attention, ne pas confondre avec les jours de présence obligatoire sur site)? "
        Q_TT_NOMBRE+=f" Si tu as plusieurs réponses, affecte le maximum de ces éventuelles réponses dans une variable 'nombrejourmaxpar{v}=\d+'. Si tu ne trouves pas de valeur, retourne nombrejourmaxpar{v}=0"
        Q_TT_NOMBRE+=f" Ta réponse devra impérativement comporter la chaîne de caractère 'nombrejourmaxpar{v}=' suivi du nombre demandé."
        reponse= rag_chain.invoke(Q_TT_NOMBRE)
        text+=f"Q_TT{k}_NOMBRE:"+reponse["text"] + "\n"
        dict_reponse[f"TT{k}_NOMBRE"]=guess_reponse_nombre(reponse,pattern=f"nombrejourmaxpar{v}")
    return dict_reponse,text

In [292]:
VAR_TRANSLATION={'DISPOSPERQTHENCEINTE': "dispositif_special_rqth_maternite_senior",
 'MONTANT_CAVIARDE': "montant_caviarde",
 'TELETRAVAIL_FLEX_SANS_LIMITE': "teletravail_regulier_flexible_sans_limite",
 'TT_REG_NOMBRE_FORMULE': "nombre_formules_teletravail_regulier",
 'DUREE': "duree_application",
 'REVERS': "presence_clause_reversibilite",
 'ADAPT': "periode_adaptation",
 'TTREG': "teletravail_regulier",
 'TTOCA': "teletravail_occasionnel",
 'TTEXC': "teletravail_exceptionnel",
 'TTMOIS': "mention_teletravail_par_mois",
 'TTMOIS_NOMBRE': "nombre_jours_teletravail_mois",
 'TTTRIM': "mention_teletravail_par_trimestre",
 'TTTRIM_NOMBRE': "nombre_jours_teletravail_trimestre",
 'TTANNEE': "mention_teletravail_par_annuel",
 'TTANNEE_NOMBRE': "nombre_jours_teletravail_annuel",
 'TTSEM': "mention_teletravail_par_semaine",
 'TTSEM_NOMBRE': "nombre_jours_teletravail_semaine",
 'TTOCAMOIS': "occas_mention_teletravail_par_mois",
 'TTOCA_MOIS_NOMBRE': "occas_nombre_jours_teletravail_mois",
 'TTOCATRIM': "occas_mention_teletravail_par_trimestre",
 'TTOCA_TRIM_NOMBRE': "occas_nombre_jours_teletravail_trimestre",
 'TTOCAANNEE': "occas_mention_teletravail_par_annuel",
 'TTOCA_ANNEE_NOMBRE': "occas_nombre_jours_teletravail_annuel",
 'TTOCASEM': "occas_mention_teletravail_par_semaine",
 'TTOCA_SEM_NOMBRE': "occas_nombre_jours_teletravail_semaine",
 'TTPRESJOURMOIS': "mention_jour_presence_par_mois",
 'PRESJOURMOIS_NOMBRE': "nombre_jour_presence_par_mois",
 'TTPRESJOURTRIM': "mention_jour_presence_par_trimestre",
 'PRESJOURTRIM_NOMBRE': "nombre_jour_presence_par_trimestre",
 'TTPRESJOURANNEE': "mention_jour_presence_par_annuel",
 'PRESJOURANNEE_NOMBRE': "nombre_jour_presence_par_annuel",
 'TTPRESJOURSEM': "mention_jour_presence_par_semaine",
 'PRESJOURSEM_NOMBRE': "nombre_jour_presence_par_semaine",
 'COMPJOUR': "mention_indemnisation_journaliere",
 'COMPJOUR_NOMBRE': "indemnisation_journaliere",
 'COMPMOIS': "mention_indemnisation_mensuelle",
 'COMPMOIS_NOMBRE': "indemnisation_mensuelle",
 'COMPMOIS_BASE_NOMBRE': "indemnisation_base_mensuelle_pour_un_jour_par_semaine",
 'COMPTRIM': "mention_indemnisation_trimestrielle",
 'COMPTRIM_NOMBRE': "indemnisation_trimestrielle",
 'COMPANNEE': "mention_indemnisation_annuelle",
 'COMPANNEE_NOMBRE': "indemnisation_annuelle",
 'COMPSEM': "mention_indemnisation_semaine",
 'COMPSEM_NOMBRE': "indemnisation_semaine",
 'COMPE': "mention_indemnisation_equipement",
 'COMPE_NOMBRE': "indemnisation_equipement",
 'COMPS': "mention_indemnite_sujetion",
 'COMPS_NOMBRE': "montant_indemnite_sujetion",
 'EQUIPEMENT': "equipement_fourni"}

# Boucle

In [293]:
import glob
PARQUET_FILE_COMPARE="Accords/Donnees_Mathilde_Pesenti_TT_Pour_DEE_v2_def.parquet"
for (index,(num_dossier,accord)) in df_init.iterrows():
    print(index,num_dossier)
    with open(f"{num_dossier}.txt","w", encoding="utf-8") as file:
        file.write(accord)

    loader = TextLoader(f"{num_dossier}.txt", encoding='utf8')
    docs=loader.load()
    
    # Chunk text
    text_splitter = CharacterTextSplitter(chunk_size=5000, 
                                          chunk_overlap=100)
    chunked_documents = text_splitter.split_documents(docs)
    
    # Load chunked documents into the FAISS index
    db = FAISS.from_documents(chunked_documents, 
                              HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))
    
    retriever = db.as_retriever()
    
    rag_chain = ( 
     {"context": retriever, "question": RunnablePassthrough()}
        | llm_chain
    )
    dict_reponse,text=process_model(rag_chain,num_dossier)
    with open(f"{num_dossier}_response.txt","w", encoding="utf-8") as file:
        file.write(text)
        print(text)
    df2=pd.DataFrame.from_dict(dict_reponse,orient="index")
    df2=df2.transpose().rename(columns=VAR_TRANSLATION)
    df2.to_csv(f"{num_dossier}.csv")
    df2.to_parquet(f"{num_dossier}.parquet")

0 T09221025781


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q_TTSEM_NOMBRE:* Mon raisonnement : Je dois trouver dans le texte une indication du nombre maximal de jours de télétravail par semaine, hors mention de télétravail ponctuel ou occasionnel.
* Phrase ou paragraphe du contexte qui répond à la question : Je ne trouve pas dans le texte l'information demandée.
* Ma réponse : nombrejourmaxparsemaine=0

1 T03821007340


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q_TTSEM_NOMBRE:
* Mon raisonnement : Je dois rechercher dans le texte des indications concernant le nombre maximal de jours de télétravail possibles par semaine. Ce nombre doit être explicite et ne pas résulter d'une opération de calcul ou de proratisation. De plus, je dois m'assurer que la notion de semaine est mentionnée explicitement dans le texte.

* Phrase ou paragraphe du contexte qui répond à la question : None, the provided context does not contain any information about the maximum number of days an employee can telework regularly per week.

* Ma réponse : nombrejourmaxparsemaine=0

2 T09221024750


...

Q_TTSEM_NOMBRE:* Mon raisonnement : Je vais rechercher dans le texte tous les paragraphes qui comportent le mot-clé "semaine" et qui concernent le nombre de jour de télétravail autorisé. Ensuite, je vérifierai si le nombre de jour de télétravail est explicite dans le paragraphe et s'il est exprimé uniquement par semaine.
* Phrase ou paragraphe du contexte qui répond à la question : Je n'ai pas trouvé de phrase ou paragraphe qui corresponde à ma recherche.
* Ma réponse : nombrejourmaxparsemaine=0

En effet, après avoir parcouru l'ensemble des documents fournis, je n'ai pas trouvé de paragraphe qui mentionne explicitement le nombre de jours de télétravail autorisé par semaine.

147 T09321006745


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q_TTSEM_NOMBRE:
* Mon raisonnement : Je dois trouver dans le texte une phrase qui contienne le mot "semaine" et qui précise le nombre maximal de jours de télétravail par semaine.
* Phrase ou paragraphe du contexte qui répond à la question : "Il a été convenu, par le présent accord, d’augmenter le nombre de jours de travail à **deux jours par semaine** permettant aux salariés concernés de recourir au télétravail en application des dispositions et du régime prévu dans l’accord du 7 décembre 2016 relatif à la mise en place du télétravail applicable au sein de l’entreprise."
* Ma réponse : nombrejourmaxparsemaine=2

148 T09221026132


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Q_TTSEM_NOMBRE:
* Mon raisonnement : Je vais rechercher dans le texte les paragraphes qui comportent le mot-clé "semaine" et qui concernent le nombre de jours de télétravail autorisés. Dans ces paragraphes, je vais chercher une expression numérique qui correspond au nombre maximal de jours de télétravail autorisés par semaine.

* Phrase ou paragraphe du contexte qui répond à la question : Malheureusement, aucun paragraphe ne comporte le mot-clé "semaine" et ne précise le nombre de jours de télétravail autorisés par semaine.

* Ma réponse : nombrejourmaxparsemaine=0



In [296]:
data_files=glob.glob('*.parquet')
df = pd.concat((pd.read_parquet(f, engine = 'pyarrow') for f in data_files))
df=df.set_index("num_dossier")
df=df[df.nombre_jours_teletravail_semaine.values!=[None]]
df2=pd.read_parquet(f"{PARQUET_FILE_COMPARE}")
df2=df2[["N..Dossier","nombre_jours_teletravail_semaine"]].set_index("N..Dossier")
df_merge=df2.merge(df,how="left",left_index=True, right_index=True)
df_merge.nombre_jours_teletravail_semaine_y=df_merge.nombre_jours_teletravail_semaine_y.astype("float64")
df_merge=df_merge.fillna(0.0)
df_merge["diff"]=(df_merge.nombre_jours_teletravail_semaine_x!=df_merge.nombre_jours_teletravail_semaine_y)
print("global accuracy",1-(df_merge["diff"].sum()/df_merge.shape[0]))

df_rempli=df_merge[df_merge.nombre_jours_teletravail_semaine_x!=0.0]
nb_correct,total =df_rempli.shape[0]-df_rempli["diff"].sum(),df_rempli.shape[0]
print("accuracy on non null value",1-(df_rempli["diff"].sum()/df_rempli.shape[0]),f"; nombre correct : {nb_correct} sur {total}")

df_rempli=df_merge[df_merge.nombre_jours_teletravail_semaine_x==0.0]
nb_correct,total =df_rempli.shape[0]-df_rempli["diff"].sum(),df_rempli.shape[0]
print("accuracy on null value",1-(df_rempli["diff"].sum()/df_rempli.shape[0]),f"; nombre correct : {nb_correct} sur {total}")

global accuracy 0.7152317880794702
accuracy on non null value 0.7604166666666666 ; nombre correct : 73 sur 96
accuracy on null value 0.6363636363636364 ; nombre correct : 35 sur 55


In [297]:
df_rempli=df_merge[df_merge.nombre_jours_teletravail_semaine_x!=0.0]
df_rempli[df_rempli["diff"]]

,nombre_jours_teletravail_semaine_x,nombre_jours_teletravail_semaine_y,diff
N..Dossier,,,
T01321011053,2.0,1.0,True
T02221003288,2.0,3.0,True
T03121008885,2.0,0.0,True
T03321006775,1.0,0.0,True
T03821006956,1.0,2.0,True
T04121001532,5.0,4.0,True
T04221004283,2.0,0.0,True
T04521003587,3.0,0.0,True
T05121003353,0.5,0.0,True
